# Get context data on models
Getting this context data allows us to add extra context data to our template. It modifies what data is available in our template and can combine multiple sources of data

In [ ]:
def get_context_data(**kwargs):
    context = super().get_context_data(**kwargs)
    
    context.update({
        'key1': value1,
        "key2": value2
    })
    
    context['single_item'] = value
    return context

## Common use cases
1. Adding form instances 
2. Including user permissions
3. Adding counts and aggregations
4. Including related data
5. Adding configuration settings

Example:

In [ ]:
from django.contrib.auth.mixins import LoginRequiredMixin, UserPassesTestMixin
from django.views.generic import ListView
from django.utils import timezone
from django.db.models import Count
from datetime import timedelta

class Requests(UserPassesTestMixin, LoginRequiredMixin, ListView):
    model = MyModel
    template_name = "requests.html"
    context_object_name = "m101_requests"
    ordering = ['-request_date']
    paginate_by = 10
    
    def get_context_data(self, **kwargs):
        context = super().get_context_data(**kwargs)
        today = timezone.now().date()
        last_week = today - timedelta(days=6)
        context.update({
            'total_requests': MyModel.objects.filter(
                request_date__date=today
            ).count(),
            
            'pending_requests': MyModel.objects.filter(
                request_status=MyModel.RequestStatus.PENDING
            ).count(),
            
            'completed_requests': MyModel.objects.filter(
                request_status=MyModel.RequestStatus.COMPLETED
            ),
            
            'user_groups': [group.name for group in self.request.user.groups.all()],
            
            'status_summary': MyModel.objects.values('request_status').annotate(
                total=Count('id')
            ),
            
            'title': 'Requests',
            'current_user': self.request.user,
        })
        return context